# Preliminary Knowledge

### Rayleigh quotient
if $\boldsymbol{x}$ is an $\textbf{eigenvector}$ of a matrix $\mathbf{A}$, the its corresponding eigenvalue is given by

$\lambda = \frac{\boldsymbol{x^{t}} A\boldsymbol{x} }{\boldsymbol{x^{t}} \boldsymbol{x}}$

### Power Iteration
Calculate the dominant eigenvector (corresponding to the top-1 eigenvalue) of the hessian Matrix $\mathbf{A}$

$\boldsymbol{v}$ = norm(random($\mathbf{A}$.size()))

itrate k times:
1. calculate the matrix-by-vector product A$\boldsymbol{v}$

    $\boldsymbol{v}$ = A$\boldsymbol{v}$
     
2. Norm for the next iteration

    $\boldsymbol{v}$ = $\boldsymbol{v}$ / norm($\boldsymbol{v}$)

return $\boldsymbol{v}$

$\boldsymbol{v}$ is the dominant eigenvector.

eigenvalue is $\lambda = \boldsymbol{v^{t}} A\boldsymbol{v}$, since $\boldsymbol{v}$ is a normed vector.

From wikipedia


---
## Construct a simple linear model

input = $[x_1, x_2]^T$

linear1.weight $W = \begin{bmatrix} w_1 & w_2 \\ w_3 & w_4 \end{bmatrix}$

linear2.weight $V = \begin{bmatrix} v_1 \\ v_2 \end{bmatrix}$

prediction: $\hat y$

Ground Truth: $y$

Loss = MSE loss

$\hat y = v_1(w_1 \cdot x_1 + w_2 \cdot x_2) + v_2(w_3 \cdot x_1 + w_4 \cdot x_2)$


To facilitate the calculation, we initialize all parameters with integers.

$[x_1, x_2]^T = [1, 1]^T $

$\begin{bmatrix} w_1 & w_2 \\ w_3 & w_4 \end{bmatrix} = \begin{bmatrix} 2 & 3 \\ 4 & 1 \end{bmatrix}$

$\begin{bmatrix} v_1 \\ v_2 \end{bmatrix} = \begin{bmatrix} 2 \\ 1 \end{bmatrix}$

$y = 14 $

$ \hat y = 2*(2+3) + 1*(1+4) = 15 $

In [1]:
device = 'cpu'
cuda = True if device == 'gpu' else False

In [2]:
import torch
import torch.nn as nn

class DemoNet2(nn.Module):
    def __init__(self, num_classes=1):
        super(DemoNet2, self).__init__()
        self.linear1 = nn.Linear(2, 2, bias=False)
        self.linear2 = nn.Linear(2, num_classes, bias=False)

    def forward(self, x):
        out = self.linear1(x)
        out = self.linear2(out)

        return out
    
demo2 = DemoNet2()
linear1_weight = torch.tensor([[2., 3.], [4., 1.]])
linear2_weight = torch.tensor([2., 1.])
demo2.linear1.weight = nn.Parameter(linear1_weight)
demo2.linear2.weight = nn.Parameter(linear2_weight)
demo2.eval()
# create loss function
criterion = torch.nn.MSELoss()
input = torch.tensor([1., 1.])
target = torch.tensor([14.])

---
# Compute the Hessian Matrix

## Calculate the Hessian matrix of the loss function with respect to $\boldsymbol{W}$

$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial w_1} = 2(\hat y - y)v_1x_1  $

$\frac{\partial L}{\partial w_2} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial w_2} = 2(\hat y - y)v_1x_2  $

$\frac{\partial L}{\partial w_3} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial w_3} = 2(\hat y - y)v_2x_1  $

$\frac{\partial L}{\partial w_4} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial w_4} = 2(\hat y - y)v_2x_2  $


$\frac{\partial^2 L}{\partial w_1^2} = \frac{\partial \frac{\partial L}{\partial w_1}}{\partial w_1} = 2\frac{\partial \hat y}{\partial w_1}v_1x_1 = 2v_1^2x_1^2 = 8$

$\frac{\partial^2 L}{\partial w_1 \partial w_2} = \frac{\partial^2 L}{\partial w_2 \partial w_1}  = \frac{\partial \frac{\partial L}{\partial w_1}}{\partial w_2} = 2\frac{\partial \hat y}{\partial w_2}v_1x_1 = 2v_1^2x_1x_2 = 8$

$\frac{\partial^2 L}{\partial w_1 \partial w_3} = \frac{\partial^2 L}{\partial w_3 \partial w_1} = \frac{\partial \frac{\partial L}{\partial w_1}}{\partial w_3} = 2\frac{\partial \hat y}{\partial w_3}v_1x_1 = 2v_1v_2x_1^2 = 4$

$\frac{\partial^2 L}{\partial w_1 \partial w_4} = \frac{\partial^2 L}{\partial w_4 \partial w_1} = \frac{\partial \frac{\partial L}{\partial w_1}}{\partial w_4} = 2\frac{\partial \hat y}{\partial w_4}v_1x_1 = 2v_1v_2x_1x_2 = 4$

$\frac{\partial^2 L}{\partial w_2^2} = \frac{\partial \frac{\partial L}{\partial w_2}}{\partial w_2} = 2\frac{\partial \hat y}{\partial w_2}v_1x_2 = 2v_1^2x_2^2 = 8$

$\frac{\partial^2 L}{\partial w_2 \partial w_3} = \frac{\partial^2 L}{\partial w_3 \partial w_2}  = \frac{\partial \frac{\partial L}{\partial w_2}}{\partial w_2} = 2\frac{\partial \hat y}{\partial w_3}v_1x_2 = 2v_1 v_2 x_1 x_2 = 4$

$\frac{\partial^2 L}{\partial w_2 \partial w_4} = \frac{\partial^2 L}{\partial w_4 \partial w_2}  = \frac{\partial \frac{\partial L}{\partial w_2}}{\partial w_4} = 2\frac{\partial \hat y}{\partial w_4}v_1x_2 = 2v_1 v_2 x_2^2 = 4$

$\frac{\partial^2 L}{\partial w_3^2} = \frac{\partial \frac{\partial L}{\partial w_3}}{\partial w_3} = 2\frac{\partial \hat y}{\partial w_3}v_2x_1 = 2v_2^2x_1^2 = 2$

$\frac{\partial^2 L}{\partial w_3 \partial w_4} = \frac{\partial^2 L}{\partial w_4 \partial w_3}  = \frac{\partial \frac{\partial L}{\partial w_3}}{\partial w_4} = 2\frac{\partial \hat y}{\partial w_4}v_2x_1 = 2 v_2^2 x_1 x_2 = 2$

$\frac{\partial^2 L}{\partial w_4^2} = \frac{\partial \frac{\partial L}{\partial w_4}}{\partial w_4} = 2\frac{\partial \hat y}{\partial w_4}v_2x_2 = 2v_2^2x_2^2 = 2$

$ H_w = \begin{bmatrix} \frac{\partial^2 L}{\partial w_1^2} & \frac{\partial^2 L}{\partial w_1 \partial w_2}  & \frac{\partial^2 L}{\partial w_1 \partial w_3} & \frac{\partial^2 L}{\partial w_1 \partial w_4} \\
\frac{\partial^2 L}{\partial w_2 \partial w_1} & \frac{\partial^2 L}{\partial w_2^2} & \frac{\partial^2 L}{\partial w_2 \partial w_3} & \frac{\partial^2 L}{\partial w_2 \partial w_4} \\
\frac{\partial^2 L}{\partial w_3 \partial w_1} & \frac{\partial^2 L}{\partial w_3 \partial w_2} & \frac{\partial^2 L}{\partial^2 w_3^2}  & \frac{\partial^2 L}{\partial w_3 \partial w_4} \\
\frac{\partial^2 L}{\partial w_4 \partial w_1} & \frac{\partial^2 L}{\partial w_4 \partial w_2} & \frac{\partial^2 L}{\partial w_4 \partial w_3} & \frac{\partial^2 L}{\partial w_4^2} \end{bmatrix} 
= 
\begin{bmatrix} 8 & 8 & 4 & 4 \\ 8 & 8 & 4 & 4 \\ 4 & 4 & 2 & 2  \\ 4 & 4 & 2 & 2  \end{bmatrix} 
$

### Calculate the top-1 eigenvalue and corresponding eigenvactor

In [3]:
import numpy as np

Matrix = np.array([[8, 8, 4, 4],[8, 8, 4, 4], [4, 4, 2, 2], [4, 4, 2, 2]])
eigs, vecs = np.linalg.eig(Matrix)
index = np.argsort(eigs)[-1]

print("The top eigenvalue of H is %.4f"%np.sort(eigs)[-1])
print("The eigenvector of top-1 eigenvalue is: ", vecs[:, index])

The top eigenvalue of H is 20.0000
The eigenvector of top-1 eigenvalue is:  [-0.63245553 -0.63245553 -0.31622777 -0.31622777]


## Calculate the Hessian matrix of the loss function with respect to  $\boldsymbol{V}$

$\frac{\partial L}{\partial v_1} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial v_1} = 2(\hat y - y)(w_1x_1 + w_2x_2)  $

$\frac{\partial L}{\partial v_2} = \frac{\partial L}{\partial \hat y} \frac{\partial \hat y}{\partial v_2} = 2(\hat y - y)(w_3x_1 + w_4x_2)  $

$\frac{\partial^2 L}{\partial v_1^2} = \frac{\partial \frac{\partial L}{\partial v_1}}{\partial v_1} = 2\frac{\partial \hat y}{\partial v_1}(w_1x_1 + w_2x_2) = 2(w_1x_1 + w_2x_2)^2 = 50$

$\frac{\partial^2 L}{\partial v_1 \partial v_2} = \frac{\partial^2 L}{\partial v_2 \partial v_1}  = \frac{\partial \frac{\partial L}{\partial v_1}}{\partial v_2} = 2\frac{\partial \hat y}{\partial v_2}(w_1x_1 + w_2x_2) = 2(w_1x_1 + w_2x_2)(w_3x_1 + w_4x_2) = 50$

$\frac{\partial^2 L}{\partial v_2^2} = \frac{\partial \frac{\partial L}{\partial v_2}}{\partial v_2} = 2\frac{\partial \hat y}{\partial v_2}(w_3x_1 + w_4x_2) = 2(w_3x_1 + w_4x_2)^2 = 50$


$ H_v = \begin{bmatrix} \frac{\partial^2 L}{\partial v_1^2} & \frac{\partial^2 L}{\partial v_1 \partial v_2} \\ \frac{\partial^2 L}{\partial v_2 \partial v_1} & \frac{\partial^2 L}{\partial v_2^2} \end{bmatrix} = = 
\begin{bmatrix} 50 & 50 \\ 50 & 50   \end{bmatrix} 
$

### Calculate the top-1 eigenvalue and corresponding eigenvactor

In [4]:
Matrix = np.array([[50, 50], [50, 50]])
eigs, vecs = np.linalg.eig(Matrix)
index = np.argsort(eigs)[-1]

print("The top eigenvalue of H is %.4f"%np.sort(eigs)[-1])
print("The eigenvector of top-1 eigenvalue is: ", vecs[:, index])

The top eigenvalue of H is 100.0000
The eigenvector of top-1 eigenvalue is:  [0.70710678 0.70710678]


---
## Let's check the result of Pyhessian function

In [5]:
from pyhessian import hessian # Hessian computation

hessian_comp = hessian(demo2, criterion, data=(input, target), cuda=cuda)
top_eigenvalues, top_eigenvector = hessian_comp.eigenvalues(top_n=1, maxIter=50)

C:\Users\Administrator\anaconda3\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Administrator\anaconda3\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\torch\csrc\autograd\engine.cpp:1000.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run 

In [6]:
top_eigenvalues

[121.97093200683594]

In [7]:
top_eigenvector

[[tensor([[0.2644, 0.2644],
          [0.1374, 0.1374]]),
  tensor([0.6433, 0.6392])]]

$121.97 \approx 20 + 100$, error exists. Besides, the Pyhessian computed eigenvectors are not normalized, and there are errors in the scaling. 

For example, for V, the ground truth is [0.707, 0.707] (1:1), output of Pyhessian is [0.6433, 0.6392] (not 1:1).

Why this error happens?

1. normalization

    It normal all the random vectors for different layers in power iteration method with the sum of all random vectors.
    
    $ V = [v_1, v_2]^T$ is normalized to $V_{norm} = [\frac{v_1}{C}, \frac{v_2}{C}]^T, C = norm(v_1) + norm(v_2)$ , $v_1, v_2$ are vectors.
    
    I think the right way is $V_{norm} = [\frac{v_1}{norm(v_1)}, \frac{v_2}{norm(v_2)}]^T$
    

2. hessian_vector_product

    It utilize the official library functions of Pytorch: torch.autograd.grad(A, B, V) to computes the sum of gradients of A with respect to the B. V should be a sequence of length matching A containing the “vector” in vector-Jacobian product. We can view it as $\frac{\partial A}{\partial B}V$, suitable to compute the step-1 of power iteration ($\frac{\partial A}{\partial B}$ = hessian matrix).
    
    $A_1$ is the gradient of Loss function with respect to the $B_1$. For example, $A_1$ and $A_2$ are the $\frac{\partial L}{\partial  \boldsymbol{W}}$ and $\frac{\partial L}{\partial \boldsymbol{V}}$, and $B_1$ and $B_2$ are weights ( $\boldsymbol{W}$ and  $\boldsymbol{V}$ in the demo network).
    
    However, the A, B and V are lists (we assume the length is 2). If we directly call torch.autograd.grad(A, B, V), the result is:
    torch.autograd.grad(A, B, V) = $[\frac{\partial A_1}{\partial B_1}V_1 + \frac{\partial A_2}{\partial B_1}V_2, \frac{\partial A_1}{\partial B_2}V_1 + \frac{\partial A_2}{\partial B_2}V_2]$, actually we need  $[\frac{\partial A_1}{\partial B_1}V_1, \frac{\partial A_2}{\partial B_2}V_2]$.

---
# So I modify some auxiliary function of Pyhessian to calculate the top-1 eigenvalue of each layer.

In [8]:
def normalization(v):
    """
    normalization of a list of vectors
    return: a list of normalized vectors v
    """
    v = [x / ((torch.sum(x * y)**0.5).item() + 1e-6) for (x, y) in zip(v, v)]
    return v

# v = [torch.randn((3,3,3,3))]
# sum_ = normalization(v)
# for s in sum_:
#     print(torch.norm(s))



def get_params_grad(model):
    """
    get model parameters and corresponding gradients
    """
    params = []
    grads = []
    for param in model.parameters():
        if not param.requires_grad:
            continue
        params.append(param)
        grads.append(0. if param.grad is None else param.grad + 0.)
    return params, grads


def group_product_sum(xs, ys):
    """
    the inner product of two lists of vectors xs,ys
    :param xs:
    :param ys:
    :return:
    """
    return sum([torch.sum(x * y) for (x, y) in zip(xs, ys)])


def group_product(xs, ys):
    """
    the inner product of two lists of variables xs,ys
    :param xs:
    :param ys:
    :return: a list of the inner product of two lists of variables xs,ys
    """
    return [torch.sum(x * y) for (x, y) in zip(xs, ys)]



# if gradsH， params，v are lists，则计算的hv[j]
# hv[j] = sum(torch.autograd.grad(gradsH[i], params[j], grad_outputs=v[i], only_inputs=True, retain_graph=True) for i in len(v))
# def hessian_vector_product(gradsH, params, v):
#     """
#     compute the hessian vector product of Hv, where
#     gradsH is the gradient at the current point,
#     params is the corresponding variables,
#     v is the vector.
#     """

#     hv = torch.autograd.grad(gradsH,
#                              params,
#                              grad_outputs=v,
#                              only_inputs=True,
#                              retain_graph=True)
#     return hv


def hessian_vector_product(gradsH, params, v):
    """
    compute the hessian vector product of Hv, where
    gradsH is the gradient at the current point,
    params is the corresponding variables,
    v is the vector.
    """
    hv = []
    for i in range(len(gradsH)):
        hv.append(torch.autograd.grad(gradsH[i],
                                 params[i],
                                 grad_outputs=v[i],
                                 only_inputs=True,
                                 retain_graph=True)[0])
    return tuple(hv)

## Let's check the output of the modified functions

In [9]:
output = demo2(input)
demo2.zero_grad()
loss = criterion(output, target)
loss.backward(create_graph=True, retain_graph=True)

params, gradsH = get_params_grad(demo2)

computed_dim, top_k = 0, 1
iteration = 50
tol = 1e-3
eigenvalues = []
eigenvectors = []

while computed_dim < top_k:
    eigenvalue = None
    v = [torch.randn(p.size()).to(device) for p in params] 
    v = normalization(v)
    for i in range(iteration):
        # only compute top-1 eigenvalue, I didn't consider the orthonormal now
#         v = orthnormal(v, eigenvectors)
        demo2.zero_grad()
        Hv = hessian_vector_product(gradsH, params, v)
        tmp_eigenvalue = group_product(Hv, v)
        
        v = normalization(Hv)
        
        if eigenvalue == None:
            eigenvalue = tmp_eigenvalue
        else:
            if abs(sum(eigenvalue) - sum(tmp_eigenvalue)) / (abs(sum(eigenvalue) + 1e-6)) < tol:
                print("Break: ", i)
                break
            else:
                eigenvalue = tmp_eigenvalue      
    eigenvalues.append(eigenvalue)
    eigenvectors.append(v)
        
    
    computed_dim += 1



Break:  2


In [10]:
eigenvalues

[[tensor(20.), tensor(100.)]]

In [11]:
eigenvectors

[[tensor([[-0.6325, -0.6325],
          [-0.3162, -0.3162]]),
  tensor([0.7071, 0.7071])]]

In [12]:
torch.norm(eigenvectors[0][0])

tensor(1.0000)

## Ground Truth

In [13]:
import numpy as np

Matrix = np.array([[8, 8, 4, 4],[8, 8, 4, 4], [4, 4, 2, 2], [4, 4, 2, 2]])
eigs, vecs = np.linalg.eig(Matrix)
index = np.argsort(eigs)[-1]

print("The top eigenvalue of H is %.4f"%np.sort(eigs)[-1])
print("The eigenvector of top-1 eigenvalue is: ", vecs[:, index])

Matrix = np.array([[50, 50], [50, 50]])
eigs, vecs = np.linalg.eig(Matrix)
index = np.argsort(eigs)[-1]

print("The top eigenvalue of H is %.4f"%np.sort(eigs)[-1])
print("The eigenvector of top-1 eigenvalue is: ", vecs[:, index])

The top eigenvalue of H is 20.0000
The eigenvector of top-1 eigenvalue is:  [-0.63245553 -0.63245553 -0.31622777 -0.31622777]
The top eigenvalue of H is 100.0000
The eigenvector of top-1 eigenvalue is:  [0.70710678 0.70710678]


# The results are consistent with the true values.

### Attachments: We can check the elements of the Hessian matrix with torch.autograd.grad

In [14]:
# grads.append(0. if param.grad is None else param.grad + 0.)

# grad = torch.autograd.grad(loss, demo2.linear1.weight)[0]
output = demo2(input)
demo2.zero_grad()
loss = criterion(output, target)
loss.backward(create_graph=True, retain_graph=True)
v = torch.zeros_like(demo2.linear1.weight.grad)
v[0, 0] = 1
grad2 = torch.autograd.grad(demo2.linear1.weight.grad, demo2.linear1.weight, v)[0]
print("The 1-st row/column elements of Hessian is:", grad2.flatten())

output = demo2(input)
demo2.zero_grad()
loss = criterion(output, target)
loss.backward(create_graph=True, retain_graph=True)
v = torch.zeros_like(demo2.linear1.weight.grad)
v[0, 1] = 1
grad2 = torch.autograd.grad(demo2.linear1.weight.grad, demo2.linear1.weight, v)[0]
print("The 2-nd row/column elements of Hessian is:", grad2.flatten())

output = demo2(input)
demo2.zero_grad()
loss = criterion(output, target)
loss.backward(create_graph=True, retain_graph=True)
v = torch.zeros_like(demo2.linear1.weight.grad)
v[1, 0] = 1
grad2 = torch.autograd.grad(demo2.linear1.weight.grad, demo2.linear1.weight, v)[0]
print("The 3-rd row/column elements of Hessian is:", grad2.flatten())

output = demo2(input)
demo2.zero_grad()
loss = criterion(output, target)
loss.backward(create_graph=True, retain_graph=True)
v = torch.zeros_like(demo2.linear1.weight.grad)
v[1, 1] = 1
grad2 = torch.autograd.grad(demo2.linear1.weight.grad, demo2.linear1.weight, v)[0]
print("The 4-th row/column elements of Hessian is:", grad2.flatten())

The 1-st row/column elements of Hessian is: tensor([8., 8., 4., 4.])
The 2-nd row/column elements of Hessian is: tensor([8., 8., 4., 4.])
The 3-rd row/column elements of Hessian is: tensor([4., 4., 2., 2.])
The 4-th row/column elements of Hessian is: tensor([4., 4., 2., 2.])
